<h1>Language Model Using Tensorflow & keras<h1> 

<h5>Importing Libraries<h5>

In [25]:
from numpy import array
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding

<h5>Preprocessing Data<h5>

In [22]:
class Preprocessing():
    
    def __init__(self,input_file):
        self.input_data_file = input_file
        self.data = None
        self.vocab_size = None
        self.encoded_data = None
        self.max_length = None
        self.sequences = None
        self.x = None
        self.y = None
        self.tokenizer = None
    
    def load_data(self):
        fp = open(self.input_data_file,'r')
        self.data = fp.read().splitlines()        
        fp.close()
        
    def encode_data(self):
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.data)
        self.encoded_data = self.tokenizer.texts_to_sequences(self.data)
        self.vocab_size = len(self.tokenizer.word_counts)+1
        
    def generate_sequence(self):
        seq_list = list()
        for item in self.encoded_data:
            l = len(item)
            for id in range(1,l):
                seq_list.append(item[:id+1])
        self.max_length = max([len(seq) for seq in seq_list])
        self.sequences = pad_sequences(seq_list, maxlen=self.max_length, padding='pre')
        self.sequences = array(self.sequences)
            
    def get_data(self):
        self.x = self.sequences[:,:-1]
        self.y = self.sequences[:,-1]
        self.y = to_categorical(self.y,num_classes=self.vocab_size)

pr = Preprocessing('data.txt')
pr.load_data()
pr.encode_data()
pr.generate_sequence()
pr.get_data()

<h3>Model

In [26]:
class Model():
    def __init__(self,params):
        self.model = None
        self.x = None
        self.y = None
        self.vocab_size = params['vocab_size']
        self.max_len = params['max_len']
        self.activation = params['activation']
        self.optimizer = params['optimizer']
        self.epochs = params['epochs']
        self.metrics = params['metrics']
        
        
    def create_model(self):
        self.model = Sequential()
        self.model.add(Embedding(self.vocab_size,10,input_length=self.max_len-1))
        self.model.add(LSTM(50))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(self.vocab_size,activation=self.activation))
        self.model.compile(loss='categorical_crossentropy',optimizer=self.optimizer,metrics=self.metrics)
        
    def run(self):
        self.model.fit(self.x,self.y,epochs=self.epochs)
    
params = {"activation":"softmax","epochs":500,"verbose":2,"loss":"categorical_crossentropy",
          "optimizer":"adam","metrics":['accuracy'],"vocab_size":pr.vocab_size,"max_len":pr.max_length}
model_obj = Model(params)
model_obj.x = pr.x
model_obj.y = pr.y
model_obj.create_model()
model_obj.run()
        

Epoch 1/500
21/21 [==============================] - 1s 31ms/sample - loss: 3.0921 - acc: 0.0476
Epoch 2/500
21/21 [==============================] - 0s 311us/sample - loss: 3.0912 - acc: 0.0476
Epoch 3/500
21/21 [==============================] - 0s 395us/sample - loss: 3.0898 - acc: 0.0476
Epoch 4/500
21/21 [==============================] - 0s 425us/sample - loss: 3.0883 - acc: 0.0952
Epoch 5/500
21/21 [==============================] - 0s 674us/sample - loss: 3.0865 - acc: 0.0476
Epoch 6/500
21/21 [==============================] - 0s 563us/sample - loss: 3.0851 - acc: 0.0952
Epoch 7/500
21/21 [==============================] - 0s 474us/sample - loss: 3.0841 - acc: 0.0952
Epoch 8/500
21/21 [==============================] - 0s 423us/sample - loss: 3.0834 - acc: 0.1429
Epoch 9/500
21/21 [==============================] - 0s 384us/sample - loss: 3.0807 - acc: 0.0952
Epoch 10/500
21/21 [==============================] - 0s 329us/sample - loss: 3.0804 - acc: 0.1905
Epoch 11/500
21/21 [

Epoch 84/500
21/21 [==============================] - 0s 368us/sample - loss: 2.3616 - acc: 0.1905
Epoch 85/500
21/21 [==============================] - 0s 335us/sample - loss: 2.3532 - acc: 0.1905
Epoch 86/500
21/21 [==============================] - 0s 324us/sample - loss: 2.2992 - acc: 0.3333
Epoch 87/500
21/21 [==============================] - 0s 312us/sample - loss: 2.2515 - acc: 0.1905
Epoch 88/500
21/21 [==============================] - 0s 440us/sample - loss: 2.2916 - acc: 0.1429
Epoch 89/500
21/21 [==============================] - 0s 283us/sample - loss: 2.1893 - acc: 0.3333
Epoch 90/500
21/21 [==============================] - 0s 286us/sample - loss: 2.1761 - acc: 0.2381
Epoch 91/500
21/21 [==============================] - 0s 292us/sample - loss: 2.1773 - acc: 0.1429
Epoch 92/500
21/21 [==============================] - 0s 347us/sample - loss: 2.0836 - acc: 0.2857
Epoch 93/500
21/21 [==============================] - 0s 320us/sample - loss: 2.1079 - acc: 0.2381
Epoch 94/5

Epoch 166/500
21/21 [==============================] - 0s 397us/sample - loss: 0.8503 - acc: 0.8571
Epoch 167/500
21/21 [==============================] - 0s 495us/sample - loss: 0.8688 - acc: 0.8095
Epoch 168/500
21/21 [==============================] - 0s 313us/sample - loss: 0.8052 - acc: 0.9524
Epoch 169/500
21/21 [==============================] - 0s 416us/sample - loss: 0.8613 - acc: 0.9048
Epoch 170/500
21/21 [==============================] - 0s 309us/sample - loss: 0.8512 - acc: 0.8571
Epoch 171/500
21/21 [==============================] - 0s 324us/sample - loss: 0.9051 - acc: 0.8095
Epoch 172/500
21/21 [==============================] - 0s 450us/sample - loss: 0.8456 - acc: 0.8095
Epoch 173/500
21/21 [==============================] - 0s 388us/sample - loss: 0.8192 - acc: 0.8571
Epoch 174/500
21/21 [==============================] - 0s 305us/sample - loss: 0.8152 - acc: 0.8095
Epoch 175/500
21/21 [==============================] - 0s 420us/sample - loss: 0.8807 - acc: 0.8095


Epoch 248/500
21/21 [==============================] - 0s 455us/sample - loss: 0.5262 - acc: 0.9048
Epoch 249/500
21/21 [==============================] - 0s 304us/sample - loss: 0.4945 - acc: 0.9048
Epoch 250/500
21/21 [==============================] - 0s 406us/sample - loss: 0.4916 - acc: 0.9048
Epoch 251/500
21/21 [==============================] - 0s 411us/sample - loss: 0.4402 - acc: 0.9048
Epoch 252/500
21/21 [==============================] - 0s 407us/sample - loss: 0.4973 - acc: 0.9048
Epoch 253/500
21/21 [==============================] - 0s 343us/sample - loss: 0.4847 - acc: 0.9048
Epoch 254/500
21/21 [==============================] - 0s 331us/sample - loss: 0.5360 - acc: 0.9048
Epoch 255/500
21/21 [==============================] - 0s 285us/sample - loss: 0.4593 - acc: 0.9048
Epoch 256/500
21/21 [==============================] - 0s 389us/sample - loss: 0.4808 - acc: 0.9048
Epoch 257/500
21/21 [==============================] - 0s 422us/sample - loss: 0.4778 - acc: 0.9048


Epoch 330/500
21/21 [==============================] - 0s 363us/sample - loss: 0.3091 - acc: 0.9524
Epoch 331/500
21/21 [==============================] - 0s 406us/sample - loss: 0.3123 - acc: 0.9524
Epoch 332/500
21/21 [==============================] - 0s 391us/sample - loss: 0.3520 - acc: 0.9524
Epoch 333/500
21/21 [==============================] - 0s 426us/sample - loss: 0.2852 - acc: 1.0000
Epoch 334/500
21/21 [==============================] - 0s 317us/sample - loss: 0.2951 - acc: 0.9524
Epoch 335/500
21/21 [==============================] - 0s 378us/sample - loss: 0.3091 - acc: 0.9524
Epoch 336/500
21/21 [==============================] - 0s 371us/sample - loss: 0.3206 - acc: 0.9048
Epoch 337/500
21/21 [==============================] - 0s 364us/sample - loss: 0.2755 - acc: 0.9524
Epoch 338/500
21/21 [==============================] - 0s 497us/sample - loss: 0.2935 - acc: 0.9524
Epoch 339/500
21/21 [==============================] - 0s 443us/sample - loss: 0.2864 - acc: 0.9524


Epoch 412/500
21/21 [==============================] - 0s 453us/sample - loss: 0.2068 - acc: 1.0000
Epoch 413/500
21/21 [==============================] - 0s 361us/sample - loss: 0.2267 - acc: 1.0000
Epoch 414/500
21/21 [==============================] - 0s 344us/sample - loss: 0.2123 - acc: 0.9524
Epoch 415/500
21/21 [==============================] - 0s 305us/sample - loss: 0.1989 - acc: 1.0000
Epoch 416/500
21/21 [==============================] - 0s 470us/sample - loss: 0.2298 - acc: 0.9524
Epoch 417/500
21/21 [==============================] - 0s 390us/sample - loss: 0.2240 - acc: 0.9524
Epoch 418/500
21/21 [==============================] - 0s 336us/sample - loss: 0.2344 - acc: 0.9048
Epoch 419/500
21/21 [==============================] - 0s 430us/sample - loss: 0.2574 - acc: 0.9048
Epoch 420/500
21/21 [==============================] - 0s 395us/sample - loss: 0.2287 - acc: 1.0000
Epoch 421/500
21/21 [==============================] - 0s 347us/sample - loss: 0.2270 - acc: 0.9524


Epoch 494/500
21/21 [==============================] - 0s 355us/sample - loss: 0.1879 - acc: 0.9524
Epoch 495/500
21/21 [==============================] - 0s 282us/sample - loss: 0.2050 - acc: 0.9524
Epoch 496/500
21/21 [==============================] - 0s 370us/sample - loss: 0.1583 - acc: 0.9524
Epoch 497/500
21/21 [==============================] - 0s 407us/sample - loss: 0.1900 - acc: 0.9048
Epoch 498/500
21/21 [==============================] - 0s 328us/sample - loss: 0.1990 - acc: 0.9524
Epoch 499/500
21/21 [==============================] - 0s 384us/sample - loss: 0.1747 - acc: 0.9524
Epoch 500/500
21/21 [==============================] - 0s 443us/sample - loss: 0.1394 - acc: 1.0000


<h4>Prediction

In [27]:
class Prediction():
    def __init__(self,model,tokenizer,max_len):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_len
        
    def predict_sequnce(self,text,num_words):
        for id in range(num_words):
            encoded_data = self.tokenizer.texts_to_sequences([text])[0]
            padded_data = pad_sequences([encoded_data],maxlen = self.max_length-1,padding='pre')
            y_pred = self.model.predict_classes(padded_data)
            out_word = ''
            for word, index in self.tokenizer.word_index.items():
                if index == y_pred:
                    out_word = word
                    break
            text += ' ' + out_word
        return text
            
pred = Prediction(model_obj.model,pr.tokenizer,pr.max_length)        
print(pred.predict_sequnce("Jack and",5))
print(pred.predict_sequnce('And Jill', 3))
print(pred.predict_sequnce('fell down', 5))
print(pred.predict_sequnce('pail of', 5))

Jack and jill went up the hill
And Jill came tumbling after
fell down down and broke his crown
pail of jill tumbling after water hill
